In [3]:
import serial
import usb
import can
import crc8
import numpy as np

In [22]:
class Can:
    adress = 0x6
    command_lengths = {
        0x00:0,
        0x01:0,
        0x02:0,
        0x03:8,
        0x05:2,     #flags
        0x06:6,
        0x07:12,
        0x0D:0,
        0x10:0,
        0x12:0,
        0x14:0,
        0x15:0,
        0x30:0,
        0x31:0,
        0x33:0,
        0x34:0,
        0x35:0,
        
    }
    
    def __init__(self):
        self.recieved = []
        self.sent = []
    
    def generate_msg(self):
        return [adress]
    
    def send(self, msg, session, status="OK"):
        
        #msg = self.generate_msg()
        
        #session.send(msg)
        
        #self.sent.append(msg)
        
        pass
    
    def get_message(self, session):
    
        #let msg is a string like "100101001010100101011010101010101010101010101"
        msg = self.recieve(session)
        
        #we need to read only last
        
        
        adress = msg[20:40]
        
        command = int(msg[20:28], base=2)
        counter = int(msg[28:32], base=2)
        
        while counter:
            counter -= 1
            self.receive(session)
            
    def recieve(self, session):
        #TODO
        #test
        msg = ""
        
        while True:
            #TODO
            break
            
        self.recieved.append(msg)
        
        return msg
        

IndentationError: expected an indented block (<ipython-input-22-23ddb04dea3e>, line 23)

In [5]:
'''
unsigned char Crc8(unsigned char *pcBlock, unsigned int len)
{
    unsigned char crc = 0xFF;
    unsigned int i;

    while (len--)
    {
        crc ^= *pcBlock++;

        for (i = 0; i < 8; i++)
            crc = crc & 0x80 ? (crc << 1) ^ 0x31 : crc << 1;
    }

    return crc;
}
'''


'\nunsigned char Crc8(unsigned char *pcBlock, unsigned int len)\n{\n    unsigned char crc = 0xFF;\n    unsigned int i;\n\n    while (len--)\n    {\n        crc ^= *pcBlock++;\n\n        for (i = 0; i < 8; i++)\n            crc = crc & 0x80 ? (crc << 1) ^ 0x31 : crc << 1;\n    }\n\n    return crc;\n}\n'

In [6]:
ord("1")

49

In [7]:
def gen_crc_8(s):
    crc = 0xFF
    for i in s:
        
        if isinstance(i, str):
            i = ord(i)
        crc ^= i
        
        for i in range(8):
            crc = (crc << 1)%256 ^ 0x31 if crc & 0x80 else (crc << 1)%256
        
    return crc
    
hex(gen_crc_8("123456789"))

'0xf7'

In [8]:
def gen_crc_16(s):
    crc = 0xFFFF
    for i in s:
        
        if isinstance(i, str):
            i = ord(i) << 8
        crc ^= i
        
        for i in range(8):
            crc = (crc << 1)%65536 ^ 0x1021 if crc & 0x8000 else (crc << 1)%65536
        
    return crc
    
hex(gen_crc_16("123456789"))

'0x29b1'

In [9]:
hex(2**15)

'0x8000'

In [10]:
#все по отдельности храниться как integer
class RS:
    commands = {
        "restart":           0x01,
        "toDefaultSettings": 0x02,
        "setDateAndTime":    0x03,
    }
    
    command_lengths = {
        0x00:0,
        0x01:0,
        0x02:0,
        0x03:8,
        0x05:2,     #flags
        0x06:6,
        0x07:12,
        0x0D:0,
        0x10:0,
        0x12:0,
        0x14:0,
        0x15:0,
        0x30:0,
        0x31:0,
        0x33:0,
        0x34:0,
        0x35:0,
        
    }
    
    errors = {
        0x00:"E_OK",
        0x0D:"E_WRONG_MEMORY_MAJ",
        0x0F:"E_CMOS_ACQ_IMPOS",
        0x17:"E_SUCCES",
        0x18:"E_FAILURE",
        #calculation errors
        0xB0:"ESV_OVERFLOW",
        0xB1:"ESV_DIV_BY_ZERO",
        0xB3:"ESV_INDETERNINATE",
        0xB4:"ESV_OUT_OF_RANGE",
        0xB6:"ESV_INVALID_ARGS",
        0xB7:"ESV_WR_INVALID_DATA",
        0xB8:"ESV_OUT_OF_DATA",
        #sending/recieveing proplems
        0xF0:"UE_OK",
        0xF1:"UE_WR_CRC",
        0xF2:"UE_WR_PLEN",
        0xF3:"UE_WR_CMD",
        0xF4:"UE_FROB_CMD",
        0xF5:"UE_WR_DLEN",
        0xF6:"UE_WR_ADDR",
        0xF7:"UE_CMD_CONFL",
        0xF8:"UE_WR_REG_NUM",
        0xF9:"UE_CMD_UNRLSD",
        0xFA:"UE_URGENT_ANS",
        0xFB:"UE_WR_PKG_NUM",
        0xFC:"UE_UNALIGNED_DATA"
        
        
        
    }
    
    adress = int("A0", base=16)
        
    def __init__(self):
        sent_msgs = []
        recived_msgs = []
        
        
    def generate_msg(self, command, data=None, base=None):
        
        """ (int(or str), int(or str or None), int(or None)) -> str"""
        
        adress = self.adress
        
        if isinstance(data, str) and base != None:
            int(data, base=base)
            
        if isinstance(command, str):
            command = self.commands[command]
            
        if data:
            data_length = self.get_length(data)
        else:
            data_length = 0
        
        data_bytes = self.get_bytes(data)
            
        crc = self.generate_crc8([adress, command, data_length] + data_bytes)
        
        msg = [adress, command, data_length] + data_bytes + [crc]
            
        return msg
        
    def get_bytes(self, data):
        data_bytes = []
        while data:
            data_bytes.append(data%256)
            data //= 256
        
        return data_bytes[::-1]
        
        
    def get_length(self, data):
        
        """ int -> int. Return number of whole bytes (not bits)"""
        
        return (len(bin(data)) - 2 + 7) // 8
        
        
    
    def generate_crc8(self, data):
        crc = 0xFF
        for i in data:

            if isinstance(i, str):
                i = ord(i)
                
            crc ^= i

            for i in range(8):
                crc = (crc << 1)%256 ^ 0x31 if crc & 0x80 else (crc << 1)%256

        return crc
    
    
    def generate_crc16(s):
        crc = 0xFFFF
        for i in s:

            if isinstance(i, str):
                i = ord(i) << 8
            crc ^= i

            for i in range(8):
                crc = (crc << 1)%65536 ^ 0x1021 if crc & 0x8000 else (crc << 1)%65536

        return crc    
    
    def check_crc_8(self, data, crc):
        return self.generate_crc8(data) == crc
    
    def send(self, msg, ser=None):
        #ser.send("test")
        if not ser:
            print msg
            return
        else:
            ser.send(bytearray(msg))
            self.sent_msgs.append(msg)
    
    def receive(self, ser):
        status = "OK"
        
        array = ser.read(3)
        if array[2] < 0xF0:
            array += ser.read(array[2] + 1)
            
        else:
            array += ser.read(1)
        #array += ser.read(array[2].encode("hex"))
        
        adress  = array[0]
        command = array[1]
        length  = array[2]
        data    = array[3:-1]
        crc     = array[3:]
        
        msg = [adress, command, length] + data + [crc]
        
        if not self.check(msg[:-1], crc):
            print "CRC is not the same, error in a message"
            status = "CRC problem"
        #it this case "length" is an error
        if length > 0xF0:
            print length, self.errors[length]
            status = "Error"
        
        self.recived_msgs.append(msg)
        
        
        return msg, status


In [11]:
rs = RS()

ans = rs.generate_msg("restart")
assert [hex(x)[2:].upper() for x in ans] == ['A0', '1', '0', '13']

ans = rs.generate_msg("setDateAndTime", data=0x7A3EC5023E01E207)
assert [hex(x)[2:].upper() for x in ans] == ['A0', '3', '8', '7A', '3E', 'C5', '2', '3E', '1', 'E2', '7', '1C']

In [12]:
#20 bits
"0"*20

'00000000000000000000'

In [13]:
"""
with serial.Serial('/dev/ttyS1', 19200, timeout=1) as ser:
    x = ser.read()          # read one byte
    s = ser.read(10)        # read up to ten bytes (timeout)
    line = ser.readline()   # read a '\n' terminated line
"""

"\nwith serial.Serial('/dev/ttyS1', 19200, timeout=1) as ser:\n    x = ser.read()          # read one byte\n    s = ser.read(10)        # read up to ten bytes (timeout)\n    line = ser.readline()   # read a '\n' terminated line\n"

In [14]:
#for serial
"""

скорость передачи 115200 бод;
количество бит данных 8;
количество стоповых бит 1;
бит четности – четный.

"""

'\n\n\xd1\x81\xd0\xba\xd0\xbe\xd1\x80\xd0\xbe\xd1\x81\xd1\x82\xd1\x8c \xd0\xbf\xd0\xb5\xd1\x80\xd0\xb5\xd0\xb4\xd0\xb0\xd1\x87\xd0\xb8 115200 \xd0\xb1\xd0\xbe\xd0\xb4;\n\xd0\xba\xd0\xbe\xd0\xbb\xd0\xb8\xd1\x87\xd0\xb5\xd1\x81\xd1\x82\xd0\xb2\xd0\xbe \xd0\xb1\xd0\xb8\xd1\x82 \xd0\xb4\xd0\xb0\xd0\xbd\xd0\xbd\xd1\x8b\xd1\x85 8;\n\xd0\xba\xd0\xbe\xd0\xbb\xd0\xb8\xd1\x87\xd0\xb5\xd1\x81\xd1\x82\xd0\xb2\xd0\xbe \xd1\x81\xd1\x82\xd0\xbe\xd0\xbf\xd0\xbe\xd0\xb2\xd1\x8b\xd1\x85 \xd0\xb1\xd0\xb8\xd1\x82 1;\n\xd0\xb1\xd0\xb8\xd1\x82 \xd1\x87\xd0\xb5\xd1\x82\xd0\xbd\xd0\xbe\xd1\x81\xd1\x82\xd0\xb8\xc2\xa0\xe2\x80\x93 \xd1\x87\xd0\xb5\xd1\x82\xd0\xbd\xd1\x8b\xd0\xb9.\n\n'

In [15]:
ans = rs.generate_msg("restart")
[hex(x)[2:].upper() for x in ans]

['A0', '1', '0', '13']

In [16]:
date = [0x7A3EC502, 0x3E01, 0xE207]
date

[2050934018, 15873, 57863]

In [17]:
ans = rs.generate_msg("setDateAndTime", data=0x7A3EC5023E01E207)
[hex(x)[2:].upper() for x in ans]

['A0', '3', '8', '7A', '3E', 'C5', '2', '3E', '1', 'E2', '7', '1C']

In [35]:
#can example
from __future__ import print_function
import can


def send_one():
    bus = can.interface.Bus(bustype='pcan', channel='PCAN_USBBUS1', bitrate=250000)
    #bus = can.interface.Bus(bustype='ixxat', channel=0, bitrate=250000)
    #bus = can.interface.Bus(bustype='vector', app_name='CANalyzer', channel=0, bitrate=250000)

    msg = can.Message(arbitration_id=0xc0ffee,
                      data=[0, 25, 0, 1, 3, 1, 4, 1],
                      extended_id=True)
    try:
        bus.send(msg)
        print("Message sent on {}".format(bus.channel_info))
    except can.CanError:
        print("Message NOT sent")

if __name__ == "__main__":
    send_one()

OSError: libpcanbasic.so: cannot open shared object file: No such file or directory

In [21]:
rs = RS()
cn = Can()

session = None
with serial.Serial(port="COM3", baudrate=115200, bytesize=8) as ser: #and can open session
    while True:
        command, data = cn.get_massage(session) #can can give us command and data
                                      #can session should be given
        
        rs.send(rs.generate_msg(command, data), ser)
        
        msg, status = rs.receive(ser)
        
        if status == "OK":
            cn.send(msg[3:-1], session)
        elif status == "Error":
            cn.send(msg[2], session, status="error")
        else:
            cn.send("", session, status="crc")
        
        


NameError: name 'Can' is not defined